In [1]:
import kagglehub
import numpy as np
import pandas as pd

from tabulate import tabulate
import matplotlib.pyplot as plt

from wordcloud import WordCloud
import seaborn as sns
from textblob import TextBlob

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import string
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Preprocessing

In [2]:
# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

# load datasets
df=pd.read_csv(path+'/WELFake_Dataset.csv')
df.head()

Path to dataset files: C:\Users\nikhi\.cache\kagglehub\datasets\saurabhshahane\fake-news-classification\versions\77


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [3]:
df.shape

(72134, 4)

In [4]:
df['article'] = df['title'] + df['text']
df = df[['article', 'label']]
df.head()

,article,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,NaN,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
4,SATAN 2: Russia unvelis an image of its terrif...,1


In [5]:
df.dropna(inplace=True)
df.drop_duplicates(subset='article', inplace=True)
df.shape

(63121, 2)

In [6]:
df['sentiment_polarity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df['sentiment_subjectivity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)

In [7]:
df.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,0.034641,0.296824
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,0.190402,0.512798
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0,0.111493,0.387106
4,SATAN 2: Russia unvelis an image of its terrif...,1,0.013342,0.403577
5,About Time! Christian Group Sues Amazon and SP...,1,-0.100461,0.428571


In [8]:
df['word_count'] = df['article'].apply(lambda x: len(str(x).split()))
df['char_count'] = df['article'].apply(lambda x: len(str(x)))
df['avg_word_length'] = df['char_count'] / (df['word_count'] + 1)
df['sentence_count'] = df['article'].apply(lambda x: len(str(x).split('.')))
df['unique_word_count'] = df['article'].apply(lambda x: len(set(str(x).split())) / len(str(x).split()))

In [10]:
import textstat
df['flesch_reading_ease'] = df['article'].apply(lambda x: textstat.flesch_reading_ease(str(x)))
df['exclamation_count'] = df['article'].apply(lambda x: str(x).count('!'))
df['question_count'] = df['article'].apply(lambda x: str(x).count('?'))

In [11]:
df.head()

,article,label,sentiment_polarity,sentiment_subjectivity,word_count,char_count,avg_word_length,sentence_count,unique_word_count,flesch_reading_ease,exclamation_count,question_count
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,0.034641,0.296824,888,5179,5.825647,55,0.494369,72.56,2,7
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,0.190402,0.512798,52,353,6.660377,2,0.942308,45.09,1,0
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0,0.111493,0.387106,1336,8115,6.069559,61,0.520210,56.59,0,1
4,SATAN 2: Russia unvelis an image of its terrif...,1,0.013342,0.403577,344,2011,5.828986,12,0.584302,51.21,0,0
5,About Time! Christian Group Sues Amazon and SP...,1,-0.100461,0.428571,256,1608,6.256809,12,0.648438,59.94,2,0


In [12]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    text = re.sub(r'\b([a-zA-Z])\.(?=[a-zA-Z])', r'\1specialdot', text)  # Preserve acronyms
    text = re.sub(r'\b([a-zA-Z])\.(?=\s|$)', r'\1specialdot', text)
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.replace("specialdot", ".")
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize, remove stopwords, and lemmatize
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    stop_words.discard("not")
    stop_words.discard("no")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Vectorize text
def vectorize(text, maxFeats, ngram):
    vectorizer = TfidfVectorizer(max_features=maxFeats, ngram_range=ngram, stop_words='english')
    X_tfidf = vectorizer.fit_transform(text)
    return X_tfidf, vectorizer

# Load data
X_train, X_test, y_train, y_test = train_test_split(
    df['article'], df['label'], 
    test_size=0.2, random_state=42, stratify=df['label']
)

# Preprocess data
X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)

# Vectorize data
X_train_vectorized, vectorizer = vectorize(X_train_processed, 2000, (1,1))
X_test_vectorized = vectorizer.transform(X_test_processed)

# Train SVM Model

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

def run_svm_pipeline(X_train, X_test, y_train, y_test):
    """
    Implements an SVM classifier for fake news detection
    """
    # Initialize SVM with a linear kernel
    svm_model = SVC(kernel='linear', probability=True, random_state=42)
    
    # Train model on the training data
    svm_model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = svm_model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    return svm_model, accuracy, report

# Run the SVM pipeline
model, accuracy, report = run_svm_pipeline(X_train_vectorized, X_test_vectorized, y_train, y_test)

# Display accuracy and report:
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)


Accuracy: 93.67%
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      6959
           1       0.93      0.93      0.93      5666

    accuracy                           0.94     12625
   macro avg       0.94      0.94      0.94     12625
weighted avg       0.94      0.94      0.94     12625



In [ ]:
# Cross-validation with GridSearchCV
from sklearn.model_selection import GridSearchCV

def run_svm_pipeline_cv(X_train, y_train):
    """
    Implements an SVM classifier for fake news detection
    """
    svm_model = SVC(kernel='linear', probability=True, random_state=42)
    
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['linear', 'rbf', 'poly']
    }
    
    grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    
    grid_search.fit(X_train, y_train)
    
    return grid_search

grid_search = run_svm_pipeline_cv(X_train_vectorized, y_train)